# prerequisite

In [2]:
# !pip install opencv-python mediapipe matplotlib

# Import Packages

In [3]:
import os
import cv2
import mediapipe as mp
import numpy as np
from matplotlib import pyplot as plt

# Functions

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(
        image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
        mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
    )
    mp_drawing.draw_landmarks(
        image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
        mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
        mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
    ) 
    mp_drawing.draw_landmarks(
        image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
    ) 
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [5]:
def test():
    cap = cv2.VideoCapture(0)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            _, frame = cap.read()

            image, results = mediapipe_detection(frame, holistic)
            draw_landmarks(image, results)
        
            cv2.putText(image, 'Action, 0', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 4, cv2.LINE_AA)
            cv2.imshow('OpenCV Feed', image)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

In [6]:
def collect_keypoints(actions, no_sequences):
    cap = cv2.VideoCapture(0)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        
        # Start collection
        while True:
            _, frame = cap.read()
            image, results = mediapipe_detection(frame, holistic)
            draw_landmarks(image, results)
            
            cv2.putText(image, 'PRESS SPACE TO START', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
            cv2.imshow('OpenCV Feed', image)
            
            key = cv2.waitKey(10) & 0xFF
            if key == ord('q'):
                return
            if key == ord(' '):
                break
        
        # Loop through actions
        for action in actions:
            new_folder = {}
            new_folder[action] = []
            
            ACTION_PATH = os.path.join(DATA_PATH, action)
            if os.path.exists(ACTION_PATH):
                dirlist = os.listdir(ACTION_PATH)
                if '.ipynb_checkpoints' in dirlist:
                    dirlist.remove('.ipynb_checkpoints')
                if '.DS_Store' in dirlist:
                    dirlist.remove('.DS_Store')
                dirmax = np.max(np.array(dirlist).astype(int))
            else:
                os.makedirs(ACTION_PATH)
                dirmax = 74

            # Loop through sequences
            for sequence in range(1,no_sequences+1):
                new_folder[action].append(dirmax+sequence)
                
                if not os.path.exists(os.path.join(ACTION_PATH, str(dirmax+sequence))):
                    os.makedirs(os.path.join(ACTION_PATH, str(dirmax+sequence)))

                # Collecting data
                for frame_num in range(sequence_length):
                    _, frame = cap.read()
                    image, results = mediapipe_detection(frame, holistic)
                    draw_landmarks(image, results)

                    # Wait logic
                    if frame_num == 0:
                        cv2.putText(image, 'STARTING COLLECTION', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                        cv2.putText(image, f'{action}, {dirmax+sequence}, {frame_num}', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 4, cv2.LINE_AA)
                        cv2.imshow('OpenCV Feed', image)
                        
                        # Wait 3 secs to start
                        key = cv2.waitKey(1000) & 0xFF
                        if key == ord('q'):
                            print(f"Interrupted, {action} {dirmax+sequence}")
                            return
                    else: 
                        cv2.putText(image, f'{action}, {dirmax+sequence}, {frame_num}', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 4, cv2.LINE_AA)
                        cv2.imshow('OpenCV Feed', image)

                    # Export keypoints
                    keypoints = extract_keypoints(results)
                    npy_path = os.path.join(ACTION_PATH, str(dirmax+sequence), str(frame_num))
                    np.save(npy_path, keypoints)

                    # Wait logic
                    key = cv2.waitKey(1) & 0xFF
                    if key == ord('q') or key == 6:
                        print(f"Interrupted, {action} {dirmax+sequence}")
                        return
                    
            print("Data Collected")
            print("-"*30)
            for key, value in new_folder.items():
                print(key, *value)

    cap.release()
    cv2.destroyAllWindows()


# Main Part

In [7]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [8]:
DATA_PATH = os.path.join('TESTDATA_0-94')
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)
sequence_length = 30

## 要修改的地方

1. `actions`: 手語名稱
2. `no_sequences`: 錄製次數
    
## Run Code

1. 一開始可以調整姿勢，準備好之後按 `空白鍵` 開始錄製資料
2. 每筆資料開始錄製之前會有 `START COLLECTION` 的標示，這時候還沒開始錄製，先把手放到定點，等字消失就開始錄製資料
3. 若中途有什麼問題，可以按 `Q` 暫停，刪掉錯誤的資料之後再繼續

In [18]:
# 要修改的地方
# record_list = ['i', 'need', 'help', 'children', 'open', 'bank', 'account', 'savings', 'have']
record_list = ['i', 'money', 'take', 'saving_book', '1', '2', '3', '4', '5',
               '6', '7', '8', '9', '0', 'taiwan', 'is', 'ten_thousand', 'id_card',
               'give_you', 'good', 'check', 'yes', 'sign', 'finish']
actions = np.array(['yes'])
no_sequences = 20
# Run Code
collect_keypoints(actions, no_sequences)

Data Collected
------------------------------
yes 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94


In [10]:
# import os

# data_folder = 'TESTDATA/9'  # DATA文件夹路径

# # 文件夹名称的映射关系
# name_mapping = {
#     '0 - 複製': '40',
#     '1 - 複製': '41',
#     '2 - 複製': '42',
#     '3 - 複製': '43',
#     '4 - 複製': '44',
#     '5 - 複製': '45',
#     '6 - 複製': '46',
#     '7 - 複製': '47',
#     '8 - 複製': '48',
#     '9 - 複製': '49',
#     '10 - 複製': '50',
#     '11 - 複製': '51',
#     '12 - 複製': '52',
#     '13 - 複製': '53',
#     '14 - 複製': '54',
#     '15 - 複製': '55',
#     '16 - 複製': '56',
#     '17 - 複製': '57',
#     '18 - 複製': '58',
#     '19 - 複製': '59',
#     '20 - 複製': '60',
#     '21 - 複製': '61',
#     '22 - 複製': '62',
#     '23 - 複製': '63',
#     '24 - 複製': '64',
#     '25 - 複製': '65',
#     '26 - 複製': '66',
#     '27 - 複製': '67',
#     '28 - 複製': '68',
#     '29 - 複製': '69',
#     '30 - 複製': '70',
#     '31 - 複製': '71',
#     '32 - 複製': '72',
#     '33 - 複製': '73',
#     '34 - 複製': '74',
# }

# # 遍历DATA文件夹中的所有项
# for item_name in os.listdir(data_folder):
#     item_path = os.path.join(data_folder, item_name)  # 获取项的完整路径

#     # 检查项是否为文件夹，并且名称在映射关系中
#     if os.path.isdir(item_path) and item_name in name_mapping:
#         new_item_name = name_mapping[item_name]  # 获取新的文件夹名称
#         new_item_path = os.path.join(data_folder, new_item_name)  # 新的文件夹路径

#         # 使用rename()函数重命名文件夹
#         os.rename(item_path, new_item_path)

# print("文件夹名称已成功更改。")

In [11]:
# import os

# data_folder = 'TESTDATA_0-74/yes'  # DATA文件夹路径

# # 文件夹名称的映射关系 5/6/7/8/9/check/good/i/take/ten_thousand
# name_mapping = {
#     '55 - 複製': '75',
#     '56 - 複製': '76',
#     '57 - 複製': '77',
#     '58 - 複製': '78',
#     '59 - 複製': '79',
#     '60 - 複製': '80',
#     '61 - 複製': '81',
#     '62 - 複製': '82',
#     '63 - 複製': '83',
#     '64 - 複製': '84',
#     '65 - 複製': '85',
#     '66 - 複製': '86',
#     '67 - 複製': '87',
#     '68 - 複製': '88',
#     '69 - 複製': '89',
#     '70 - 複製': '90',
#     '71 - 複製': '91',
#     '72 - 複製': '92',
#     '73 - 複製': '93',
#     '74 - 複製': '94',
# }

# # 遍历DATA文件夹中的所有项
# for item_name in os.listdir(data_folder):
#     item_path = os.path.join(data_folder, item_name)  # 获取项的完整路径

#     # 检查项是否为文件夹，并且名称在映射关系中
#     if os.path.isdir(item_path) and item_name in name_mapping:
#         new_item_name = name_mapping[item_name]  # 获取新的文件夹名称
#         new_item_path = os.path.join(data_folder, new_item_name)  # 新的文件夹路径

#         # 使用rename()函数重命名文件夹
#         os.rename(item_path, new_item_path)

# print("文件夹名称已成功更改。")